# Senegalese tax and benefit system from scratch

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from openfisca_core.columns import IntCol, DateCol, FloatCol
from openfisca_core.formulas import set_input_divide_by_period
from openfisca_core.variables import Variable

from openfisca_senegal import SenegalTaxBenefitSystem
from openfisca_senegal.entities import Individus

In [3]:
tax_benefit_system = SenegalTaxBenefitSystem()

In [7]:
class id_famille(Variable):
    column = IntCol
    entity_class = Individus
    is_permanent = True
    label = u"Identifiant de la famille"

tax_benefit_system.update_variable(id_famille)

class role_dans_famille(Variable):
    column = IntCol
    entity_class = Individus
    is_permanent = True
    label = u"Rôle dans la famille"

tax_benefit_system.update_variable(role_dans_famille)

class date_de_naissance(Variable):
    column = DateCol
    entity_class = Individus
    label = u"Date de naissance"

tax_benefit_system.update_variable(date_de_naissance)

class salaire(Variable):
    column = FloatCol
    entity_class = Individus
    label = "Salaire"
    set_input = set_input_divide_by_period

tax_benefit_system.update_variable(salaire)

class impot_revenu(Variable):
    column = FloatCol
    entity_class = Individus
    label = u"Impôt sur le revenu"

    def function(self, simulation, period):
        salaire = simulation.calculate_add('salaire', period)
        return period, salaire * 0.3

tax_benefit_system.update_variable(impot_revenu)

tax_benefit_system

In [8]:
scenario = tax_benefit_system.new_scenario()

In [10]:
scenario.init_single_entity(
    parent1={'salaire': 100000},
    period='2015',
)

In [11]:
simulation = scenario.new_simulation()

In [13]:
simulation.calculate('salaire', '2015')

array([ 100000.], dtype=float32)

In [14]:
simulation.calculate('impot_revenu', '2015')

array([ 30000.00195312], dtype=float32)